# Federeted Learning

A technique for training Machine Learning models on the data to which you do not have access. 

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft. PySyft is extention to the major deep learning toolkits.

So, what we are going to be looking at here is a new set of tools that allows us to actually have an interface that is nearly identical to PyTorch, but allow us to execute commands on remote machines and coordinate how data is moved amongst differetn machins.

Pysyft is extention of severel deep learning libraries with new piece that makes it easier to do privacy-preserving deep learning.


In [86]:
#import torch like we normaly did
import torch as th

In [87]:
#we create torch tensor and use them in normla way
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [88]:
#import syft 
import syft as sy

**Let's create a hook: This is going to actually modify pytorch with a new functionality. We are going to pass in the references to the Torch library here. Behind the scenes it actually just modified a bunch of pytorch API. So, we can do things that we would like to do with privacy-preversing deep laerning**

In [68]:
hook = sy.TorchHook(th)

### Lesson: Basic Remote Execution in PySyft¶
The essence of federated learning is ability to train models in paralel on a wide range of machines. inorder to that, we first need to the ability to tell remote machines to execute operations required for deep learning.

Thus, instead of using torch tensors locally, we 're going to work with pointers to tensor that exist on a different machine. However in order to another machine really look like.

* **Pysyft is create this interface for us, though somethings called worker. The first thing we're going to create is called virtual worker that simulates the interface that we might have to another machines.**


In [89]:
bob = sy.VirtualWorker(hook, id='bob')

* **So, with Pysyft, we have tensors that are owned by a variety of different machines, a variety of different workers. So, these workers form another core primitive of the frame now, workers really quite something simple. it just a collection of objects**

In [90]:
bob._objects


{49949543929: tensor([1, 2, 3, 4, 5])}

In [91]:
#let's crete a data here
x = th.tensor([1,2,3,4,5])

In [92]:
#send this data to bob
x = x.send(bob)

In [93]:
#and then look bob's objets
bob._objects

{49949543929: tensor([1, 2, 3, 4, 5]), 75396543259: tensor([1, 2, 3, 4, 5])}

* in bob's collection: there is a the tensor that we just initialized was actually send to bob.

In [94]:
x


(Wrapper)>[PointerTensor | me:18885926581 -> bob:75396543259]

* **x-> returned was a pointer to the remote object. instead of executing these command locally like a normal tensor would. each command is serilized to a simple JSON or tuple format sent to Bob then Bob execute it an our Behalf, and returns to us a pointer to the new objects**

* **if we look at the x 's location. We can see that this pointer pointed to towards bob **

In [95]:
x.location


<VirtualWorker id:bob #objects:2>

In [96]:
x.location == bob

True

In [97]:
x.id

18885926581

In [98]:
x.id_at_location

75396543259

* **x has an id_at_location and idas well, These two pieces of metadata actually allow us to comminicate with Bob. When we executing command, It's going to send a message to self.location and say "Hey. Bob find the tensor that has this particular id and execute the command I would like for you to execute."**

In [99]:
x.owner

<VirtualWorker id:me #objects:0>

* **There one more attribute that all tensors and Pysyft have which is an owner. in this case because we are the client the owner defaults to be "me". This reveals one another workers which was created without our knowledge. this was created when we first imported and hooked Pysyft into Pytorch.**

In [100]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [101]:
x


(Wrapper)>[PointerTensor | me:18885926581 -> bob:75396543259]

In [102]:
#let's get back the information back from bob
x = x.get()

In [103]:
x

tensor([1, 2, 3, 4, 5])

In [104]:
#let's check the bob's collections
bob._objects

{49949543929: tensor([1, 2, 3, 4, 5])}